# Data Wrangling and Analysis with MongoDB
### OpenStreetMap Project
##### Tania Lincoln 08/20/2017

### Summary
This is preliminary investigation of the Northwest Austin custom map area.  The objective of this exercise is to clean up a portion of an open map source xml data set of 50 MB or more, format it into json, and load it into MongoDB so it can be further analyzed.  MongoDB is an opensource, no sql, document database.
<br>

### Import Libraries
Import some common libraries and use common abbreviations

In [1]:
# import important libraries
import os
import sys
from pymongo import MongoClient
import pprint

### Display Current Environment Info

In [2]:
#It's a good practice to display the environment information
# print environment variables, so others can see what we are using
print("python " + sys.version)
print("")

python 2.7.13 |Anaconda 4.3.1 (64-bit)| (default, Dec 19 2016, 13:29:36) [MSC v.1500 64 bit (AMD64)]



### Data Wrangling Explained

#### Scripts Used for Wranging
##### 1_create_sample_osm.py
This script is executed first to create a smaller osm file from the original one.  I changed the k value so I could get different data in the sample file.
##### 2a_audit_osm.py
The audit script was used to learn more about the data.  I redirected an output to a file.  When I would learn more about a portion of the data, I'd make changes to the audit script and re-execute it.
##### 3a_clean_osm.py
This script re-shape the data into a json document so it could be loaded into mongoDB.  I chose to data wrangle roads, names, state, and phone numbers.  Because many names were actually streets, I fixed the street abbreviations similar to how address streets were fixed.  State was made into abbreviations and capatalized.  Phone numbers were standardized.  I grouped all road related tags into one subdocument and created another section with in for lane information.  gnis and tiger data was put into a sub-document, so I could talk about it more in the conclusion.  I took starter code for the address changes and refactored it, so it could be used for other data cleansing efforts.

### Data Files and Sizes

In [3]:
# Convert bytes to other formats to make it more easy to read  
# Taken from: http://code.activestate.com/recipes/577081-humanized-representation-of-a-number-of-bytes/
def humanize_bytes(bytes, precision=1):
    """
    Return a humanized string representation of a number of bytes.
    >>> humanize_bytes(1)
    '1 byte'
    >>> humanize_bytes(1024)
    '1.0 kB'
    >>> humanize_bytes(1024*123)
    '123.0 kB'
    >>> humanize_bytes(1024*12342)
    '12.1 MB'
    >>> humanize_bytes(1024*12342,2)
    '12.05 MB'
    >>> humanize_bytes(1024*1234,2)
    '1.21 MB'
    >>> humanize_bytes(1024*1234*1111,2)
    '1.31 GB'
    >>> humanize_bytes(1024*1234*1111,1)
    '1.3 GB'
    """
    abbrevs = (
        (1<<50L, 'PB'),
        (1<<40L, 'TB'),
        (1<<30L, 'GB'),
        (1<<20L, 'MB'),
        (1<<10L, 'kB'),
        (1, 'bytes')
    )
    if bytes == 1:
        return '1 byte'
    for factor, suffix in abbrevs:
        if bytes >= factor:
            break
    return '%.*f %s' % (precision, bytes / factor, suffix)

In [4]:
# print out the filesizes
filesizeXML = os.path.getsize('NW_Austin.osm')
filesizeJSON = os.path.getsize('NW_Austin.osm.json')

print "NW_Austin.osm is " + humanize_bytes(filesizeXML, precision=1)
print "NW_Austin.osm.json is " + humanize_bytes(filesizeJSON, precision=1)

NW_Austin.osm is 268.0 MB
NW_Austin.osm.json is 547.0 MB


### Load Data
- Data was loaded by using the bulk import command, using the following steps:
- Make sure that Mongod, the database service, is turned on
- From the command line, navigate to the MongoDB bin directory
- To make the command easier to create, copy the json file to the bin directory
- Execute the bulk import command
    - $> mongoimport -d openMap -c nwAust --file NW_Austin.osm.json

### Explore the Data (Basic)

#### Connect to MongoDB

In [5]:
# connect to MongoDB
def get_db():
    from pymongo import MongoClient
    client = MongoClient('mongodb://127.0.0.1:27017')
    db = client.openMap
    return db

# create the key data objects we'll need to analysis
db = get_db()
collection = db.nwAustin

In [6]:
# create an generic aggregation function
def aggregate_collection(collection, pipeline):
    return [doc for doc in collection.aggregate(pipeline)]

#### Some Examples of Documents in the Collection

Generic Document

In [7]:
docs = collection.find().limit(1)
for doc in docs:
    pprint.pprint(doc)

{u'_id': ObjectId('599a037b4bdb172147b55dd4'),
 u'created': {u'changeset': u'8497118',
              u'timestamp': u'2011-06-20T18:36:15Z',
              u'uid': u'388279',
              u'user': u'Tylan',
              u'version': u'15'},
 u'id': u'26546004',
 u'pos': [u'-97.7972587', u'30.4695355'],
 u'type': u'node'}


Road Document

In [8]:
docs = collection.find({'road':{'$exists' :1}}).limit(1)
for doc in docs:
    pprint.pprint(doc)

{u'_id': ObjectId('599a037b4bdb172147b55dd7'),
 u'created': {u'changeset': u'13420621',
              u'timestamp': u'2012-10-09T01:08:42Z',
              u'uid': u'119881',
              u'user': u'claysmalley',
              u'version': u'28'},
 u'id': u'26546008',
 u'pos': [u'-97.7966751', u'30.469115'],
 u'road': {u'highway': u'traffic signals'},
 u'type': u'node'}


Amentity Document

In [9]:
docs = collection.find({'amenity':{'$exists' :1}}).limit(1)
for doc in docs:
    pprint.pprint(doc)

{u'_id': ObjectId('599a037c4bdb172147b5f601'),
 u'amenity': u'fuel',
 u'created': {u'changeset': u'21599243',
              u'timestamp': u'2014-04-10T03:02:34Z',
              u'uid': u'703517',
              u'user': u'Iowa Kid',
              u'version': u'3'},
 u'id': u'340469022',
 u'name': u'Exxon',
 u'pos': [u'-97.7417023', u'30.4435595'],
 u'type': u'node'}


Problem Docment (Tiger)

In [10]:
docs = collection.find({'tiger':{'$exists' :1}}).limit(1)
for doc in docs:
    pprint.pprint(doc)

{u'_id': ObjectId('599a03a04bdb172147c8ebea'),
 u'created': {u'changeset': u'44631401',
              u'timestamp': u'2016-12-23T19:14:23Z',
              u'uid': u'3747719',
              u'user': u'cameron398',
              u'version': u'54'},
 u'id': u'4531227',
 u'name': u'Ranch Road 620',
 u'node_ref': [u'28110282',
               u'1332549604',
               u'2081247373',
               u'2234484687',
               u'2081247421',
               u'2234484705',
               u'4569243745',
               u'320892904',
               u'4346022134',
               u'28110282',
               u'1332549604',
               u'2081247373',
               u'2234484687',
               u'2081247421',
               u'2234484705',
               u'4569243745',
               u'320892904',
               u'4346022134',
               u'28110282',
               u'1332549604',
               u'2081247373',
               u'2234484687',
               u'2081247421',
               u'22344

#### Count of documents in the collection

In [11]:
count = collection.find().count()
print "There are " + str(count) + " documents in the collection"

There are 1402041 documents in the collection


#### Count of nodes

In [12]:
count = collection.find({'type':'node'}).count()
print "There are " + str(count) + " nodes"

There are 1280258 nodes


#### Count of ways

In [13]:
count = collection.find({'type':'way'}).count()
print "There are " + str(count) + " ways"

There are 121782 ways


#### What other types are there?

In [14]:
pprint.pprint(db.nwAustin.distinct('type'))

[u'node', u'way', u'multipolygon']


#### Count of multipolygon

In [15]:
count = collection.find({'type':'multipolygon'}).count()
print "There are " + str(count) + " multipolygons"

There are 1 multipolygons


#### Distinct User Count

In [16]:
count_uid = len(collection.distinct('created.uid'))
count_user = len(collection.distinct('created.user'))
if count_uid == count_user:
    print "Users and User ID have the same count \n"
    print "There are " + str(count_uid) + " distinct users"
else:
    print "User and User IDs have a different count\n"
    print "There are " + str(count_uid) + " distinct user IDs and " + str(count_user) + " user names" 

Users and User ID have the same count 

There are 478 distinct users


### Explore the Data (more complex questions)

##### Question:  Who are the top 5 contributers?

In [17]:
group = {"$group":{"_id":"$created.user", "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}
limit = {"$limit":10}
pipeline = [group, sort, limit]
pprint.pprint(aggregate_collection(collection, pipeline))

[{u'_id': u'ccjjmartin_atxbuildings', u'count': 581431},
 {u'_id': u'ccjjmartin__atxbuildings', u'count': 424789},
 {u'_id': u'patisilva_atxbuildings', u'count': 89956},
 {u'_id': u'kkt_atxbuildings', u'count': 65767},
 {u'_id': u'wilsaj_atxbuildings', u'count': 64101},
 {u'_id': u'richlv', u'count': 22007},
 {u'_id': u'woodpeck_fixbot', u'count': 18010},
 {u'_id': u'HJD', u'count': 16562},
 {u'_id': u'ELadner', u'count': 12631},
 {u'_id': u'lyzidiamond_atxbuildings', u'count': 10902}]


ccjjmartin is the highest contributor.  This persons login is split amongst two names and two ids.  Biggest contributors overall come from atxbuildings.

##### Question:  When were the last few contribution made?  How many changes were submitted?

In [18]:
group = {"$group":{"_id":"$created.timestamp", "count":{"$sum":1}}}
sort = {"$sort":{"_id":-1}}
limit = {"$limit":10}
pipeline = [group, sort, limit]
pprint.pprint(aggregate_collection(collection, pipeline))

[{u'_id': u'2017-08-05T07:15:33Z', u'count': 1},
 {u'_id': u'2017-08-04T15:59:56Z', u'count': 1},
 {u'_id': u'2017-08-04T06:21:02Z', u'count': 1},
 {u'_id': u'2017-08-02T21:32:12Z', u'count': 10},
 {u'_id': u'2017-08-02T21:32:11Z', u'count': 7},
 {u'_id': u'2017-08-02T21:32:10Z', u'count': 22},
 {u'_id': u'2017-08-02T21:32:09Z', u'count': 40},
 {u'_id': u'2017-08-02T21:32:08Z', u'count': 9},
 {u'_id': u'2017-08-02T21:32:07Z', u'count': 13},
 {u'_id': u'2017-08-02T21:32:06Z', u'count': 12}]


Changes have been recently made!  The date was difficult to handle.  I would have liked to just have dates instead of timestamps to see what the trending over time has been.  Austin has a lot of new construction and changes, I wanted to see if this was reflected in the inserted dates.

##### Question:  What amenities are in the area

In [19]:
match = {"$match":{"amenity":{"$exists":1}}}
group = {"$group":{"_id":"$amenity", "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}
limit = {"$limit":10}
pipeline = [match, group, sort, limit]
pprint.pprint(aggregate_collection(collection, pipeline))

[{u'_id': u'parking', u'count': 439},
 {u'_id': u'restaurant', u'count': 194},
 {u'_id': u'fast food', u'count': 149},
 {u'_id': u'school', u'count': 87},
 {u'_id': u'fuel', u'count': 81},
 {u'_id': u'place of worship', u'count': 67},
 {u'_id': u'bank', u'count': 44},
 {u'_id': u'waste basket', u'count': 42},
 {u'_id': u'bench', u'count': 33},
 {u'_id': u'cafe', u'count': 29}]


Wow, lots of parking in NW Austin.  Other than parking being first, the top half doesn't surprise me.  NW Austin is very suburban, lots of infrastructure for families and homes.

##### Question:  What type of fast food and resturants does NW Austin like?

In [20]:
match = {"$match":{"amenity":{"$exists":1}, "amenity": {"$in" :["fast food", "restaurant", "cafe"]}}}
group = {"$group":{"_id":"$cuisine", "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}
limit = {"$limit":10}
pipeline = [match, group, sort, limit]
pprint.pprint(aggregate_collection(collection, pipeline))

[{u'_id': None, u'count': 215},
 {u'_id': u'burger', u'count': 22},
 {u'_id': u'mexican', u'count': 21},
 {u'_id': u'sandwich', u'count': 14},
 {u'_id': u'pizza', u'count': 12},
 {u'_id': u'chinese', u'count': 9},
 {u'_id': u'american', u'count': 8},
 {u'_id': u'italian', u'count': 8},
 {u'_id': u'sushi', u'count': 8},
 {u'_id': u'coffee shop', u'count': 7}]


Unfortunately a lot of the resturants are not classified.  Because it's Austin, I would suspect a lot of Mexican and Tex-Mex resturants - we love our tacos!  However, since it's NW Austin, I would have suspected more cuisines from asian countries too.  This list looks fairly typical with burgers, sandwhiches, and pizza in the top.

##### Question:  What type of road ways are there?  Do cars rule or is there room for pedestrians and bikes?

In [21]:
match = {"$match":{"road":{"$exists":1}}}
project = {"$project":{"road_type":["$road.mode", "$road.highway"]}}
unwind = {"$unwind": "$road_type"}
group = {"$group":{"_id":"$road_type", "count":{"$sum":1}}}
match2 = {"$match":{"_id":{"$in":[None, "footway", "path", "[sidewalk]", 
                                  "residential", "unclassified", "cycleway", "[cycleway]", "pedestrian"]}}}
sort =  {"$sort":{"count":-1}}
limit = {"$limit":100}
pipeline = [match, project, unwind, group, match2, sort, limit]
pprint.pprint(aggregate_collection(collection, pipeline))

[{u'_id': None, u'count': 17108},
 {u'_id': u'residential', u'count': 4493},
 {u'_id': u'footway', u'count': 376},
 {u'_id': u'unclassified', u'count': 108},
 {u'_id': u'path', u'count': 79},
 {u'_id': u'cycleway', u'count': 20},
 {u'_id': [u'cycleway'], u'count': 12},
 {u'_id': u'pedestrian', u'count': 3}]


There are numerous places for other modes of transportation, however roadways for cars dominate.  There are also very few dedicated cycleways.  However, many residential streets would serve all purposes.  This is pretty tricky to investigate, the data would need much more cleansing.

### Conclusions

I had a really hard time parsing dates using the aggregation pipeline.  They are formatted as follows:  '2017-08-05T07:15:33Z'.  Given this, I would have transformed the data before loading it into mongoDB.  I wanted to investigate whether changes have spiked in the last few years because of growth.  

The tiger and gnis data seemed really subdivided and repetitive to other sections.  For example, name would often have a street address when a node was describing a road.  The same name would be represented in 3 sections for the tiger data.  I was going to ignore all of it, but decided to keep it for my conclusion.  I created a sub-document for each.

Here is an example of how Swan Drive appears in the name and how it appears in the tiger parts.

u'name': u'Swan Drive',
  u'tiger': {u'cfcc': u'A41',
             u'county': u'Travis, TX',
             u'name_base': u'Swan',
             u'name_type': u'Dr',
             u'reviewed': u'no',
             u'zip_left': u'78750',
             u'zip_right': u'78750'},
            
Another issue with the tiger data was the zip codes.  There was a zip_left and a zip_right. In my data investigation, there seemed no difference. 

#### Other ideas about the dataset

##### API integration with yelp or creating our own map-yelp

We can also use activity sites APIs like yelp or opentable to audit and source data. It's more upto date than the yellow pages and driven by a community of active users.  Ratings and Closed statuses could help drive relevance and cleanup.

Like there are trusted reviews on yelp.  We could also add ratings to the quality of open map data added by a user and have a leaderboard for how manu submission or fixes they've made.

An exteral app, like yelp, could be created for other map features.  Where users could tag map features features like the best rush hour route, the hilliest road run, best scenic outlook, or my favorite motorcycle drive.  We can use this personalization to create a community.

In [59]:
count = collection.find({'name':'Block Buster'}).count()
print "There is still " + str(count) + " BlockBuster.  They all closed in 2008-ish."

There is still 1 BlockBuster.  They all closed in 2008-ish.


Yelpers Report it as being closed.
- https://www.yelp.com/biz/blockbuster-video-austin-21

These are newer resturants in the area, which have not been included in openMaps yet.

In [60]:
print "Here are some new places that haven't been included into OpenMaps yet.\n"

count = collection.find({"name" : {"$regex" : ".*[b|B]aretto.*"}}).count()
if count == 0:
    print "Baretto is missing"
else:
    print "Baretto is included"
    
count = collection.find({"name" : {"$regex" : ".*[k|K]anji.*"}}).count()
if count == 0:
    print "Kanji Ramen is missing"
else:
    print "Kanji Ramen is included"
    
count = collection.find({"name" : {"$regex" : ".*[p|P]our.*"}}).count()
if count == 0:
    print "Pour House is missing"
else:
    print "Pour House is included"

Here are some new places that haven't been included into OpenMaps yet.

Baretto is missing
Kanji Ramen is missing
Pour House is missing


- https://www.yelp.com/biz/baretto-austin
- https://www.yelp.com/biz/kanji-ramen-austin
- https://www.yelp.com/biz/pour-house-pints-and-pies-austin

##### Possible Challenges with integration

In order for this to work users really need to favor apps like yelp.  Tying our success to yelps success could cause reworkd down the road.  Yelp really only has businesses and data about businesses.  We'd be missing opportunities for better data in areas related to roadways or geographic features.

To creat an app similar to yelp would be undertaking a lot of new responsibilities.  Only good community engagement will make it successful.  Because I could tag a route and others could tag similar routes, we'd have the potential for duplicate data or worse having to make a choice for which data is most correct or should be included.  I think we'd run into a similar situation seen with the tiger and gnis data.

### Acknowledgements

- https://stackoverflow.com/questions/20796714/how-do-i-start-mongo-db-from-windows
- http://wiki.openstreetmap.org/wiki/USGS_GNIS
- http://wiki.openstreetmap.org/wiki/TIGER_to_OSM_Attribute_Map
- https://docs.google.com/document/d/1F0Vs14oNEs2idFJR3C_OPxwS6L0HPliOii-QpbmrMo4/pub


##### Template Version History
1.0, 06/30/2017, Created